<a href="https://colab.research.google.com/github/hassanfv/changa/blob/master/test_GPU_16_Nov_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile test1.cu
#include <iostream>
#include <ctime>
#include <thrust/sort.h>
#include <algorithm>
using namespace std;

__global__ void smoothing(float *x, float *y, float *z, float *dist, float *hres, float *h_previous, int N){
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x;

  for(int i = index; i < N; i += stride){
    if(i < N){
    for(int j = 0; j < N; j++){
      float dx = x[j] - x[i];
      float dy = y[j] - y[i];
      float dz = z[j] - z[i];
      dist[i] = sqrt(dx*dx + dy*dy + dz*dz);
    }
    // ********* MY SORTING ALGORITHM *********
    //const int M = N;
    float dist_h[10000];
    for(int j = 0; j < N; j++){
      dist_h[j] = 1.02f;
    } 
    int k = 0;
    for(int j = 0; j < N; j++){
      if(dist[j] < 3.0f * h_previous[i]){
        dist_h[k] = dist[j];
        k++;
      }
    }

    int N_h = k - 1;
    float temp = 1.5f;
    for(int m = 1; m < N_h; m++){
      for(int n = N_h - 1; n >= m; n--){
        if(dist_h[n] < dist_h[n - 1]){
          temp = dist_h[n];
          dist_h[n] = dist_h[n -1];
          dist_h[n - 1] = temp;
        }
      }
    }
    // ********* END OF MY SORTING ALGORITHM *********
    hres[i] = dist_h[64]*0.5;
    }
  }
}


int main(){

  int N = 1e4;
  srand(time(NULL));

  // Allocate Unified Memory - accessible by CPU and GPU
  float *x, *y, *z, *dist, *hres, *h_previous;
  cudaMallocManaged(&x, N * sizeof(float));
  cudaMallocManaged(&y, N * sizeof(float));
  cudaMallocManaged(&z, N * sizeof(float));

  cudaMallocManaged(&dist, N * sizeof(float));
  cudaMallocManaged(&hres, N * sizeof(float));
  
  cudaMallocManaged(&h_previous, N * sizeof(float));

  // Initializing the arrays on the host
  for(int i = 0; i < N; i++){
    x[i] = (double)rand()/RAND_MAX;
    y[i] = (double)rand()/RAND_MAX;
    z[i] = (double)rand()/RAND_MAX;

    dist[i] = 1000.0;
    hres[i] = 1000.0;
    h_previous[i] = 0.10;
  }

  int blockSize = 512;
  int gridSize = (N + blockSize - 1)/blockSize;
  smoothing<<<gridSize, blockSize>>>(x, y, z, dist, hres, h_previous, N);

  // wait for GPU to finish before accessing the host
  cudaDeviceSynchronize();
  
  thrust::sort(dist, dist + N);

  for(int i = 0; i < 70; i++){
    cout << hres[i] << endl;
  }

  cout << "*******************" << endl;
  //cout << hres[64];
  
  
}

Overwriting test1.cu


In [ ]:
%%shell
nvcc test1.cu -o test1

In [ ]:
%%writefile testx.cu
#include <algorithm>
#include <iostream>

const int N = 3;

__global__ void MatAdd(float A[][N], float B[][N], float C[][N])
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    if (i < N && j < N)
        C[i][j] = A[i][j] + B[i][j];
}

int main()
{
    float* A; cudaMallocManaged(&A, N*N*sizeof(float));
    float* B; cudaMallocManaged(&B, N*N*sizeof(float));
    float* C; cudaMallocManaged(&C, N*N*sizeof(float));

    const float A_vals[N][N]={{1,0,0},{0,1,0},{0,0,1}};
    const float B_vals[N][N]={{1,0,0},{0,1,0},{0,0,1}};
    float (*C_vals)[N] = reinterpret_cast<float (*)[N]>(C);

    std::copy(&A_vals[0][0], &A_vals[0][0] + N*N, A);
    std::copy(&B_vals[0][0], &B_vals[0][0] + N*N, B);

    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks(1, 1);
    MatAdd<<<numBlocks, threadsPerBlock>>>( reinterpret_cast<float (*)[N]>(A),
                                            reinterpret_cast<float (*)[N]>(B),
                                            C_vals );

    cudaDeviceSynchronize();

    for(int i=0; i<N; i++) {
        for(int j=0; j<N; j++) {
            std::cout << C_vals[i][j] << "  ";
        }
        std::cout << std::endl;
    }

    return 0;
}

Writing testx.cu


In [ ]:
%%shell
nvcc testx.cu -o testx

In [ ]:
%%shell
./testx

2  0  0  
0  2  0  
0  0  2  


In [34]:
%%writefile test1.cu
#include <iostream>
#include <ctime>
#include <thrust/sort.h>
#include <algorithm>
using namespace std;

__global__ void smoothing(float *dist, float *hres, int N, int i){

  int index = threadIdx.x + blockIdx.x * blockDim.x;

  if(index < N){
    float temp = 0.0f;
    for(int m = 1; m < N; m++){
      for(int n = N - 1; n >= m; n--){
        if(dist[n] < dist[n - 1]){
          temp = dist[n];
          dist[n] = dist[n -1];
          dist[n - 1] = temp;
        }
      }
    }
    //if(index == i){
    //  hres[i] = dist[64];
    hres[index] = 1.0f;
    //}
  }
}


int main(){

  int N = 1e4;
  srand(time(NULL));

  // Allocate Unified Memory - accessible by CPU and GPU
  float *x, *y, *z, *dist, *hres;
  cudaMallocManaged(&x, N * sizeof(float));
  cudaMallocManaged(&y, N * sizeof(float));
  cudaMallocManaged(&z, N * sizeof(float));

  cudaMallocManaged(&dist, N * sizeof(float));
  cudaMallocManaged(&hres, N * sizeof(float));

  // Initializing the arrays on the host
  for(int i = 0; i < N; i++){
    x[i] = (double)rand()/RAND_MAX;
    y[i] = (double)rand()/RAND_MAX;
    z[i] = (double)rand()/RAND_MAX;

    dist[i] = 1000.0;
    hres[i] = 1000.0;
  }

  for(int i = 0; i < N; i++){

    for(int j = 0; j < N; j++){
      dist[j] = sqrt((x[j]-x[i])*(x[j]-x[i]) + (y[j]-y[i])*(y[j]-y[i]) + (z[j]-z[i])*(z[j]-z[i]));
    }

    int blockSize = 256;
    int gridSize = (N + blockSize - 1)/blockSize;
    smoothing<<<gridSize, blockSize>>>(dist, hres, N, i);

    // wait for GPU to finish before accessing the host
    cudaDeviceSynchronize();
  }

  for(int i = 0; i < 10; i++){
    cout << dist[i] << endl;
  }

  cout << "*******************" << endl;
  
  
}

Overwriting test1.cu


In [35]:
%%shell
nvcc test1.cu -o test1

In [36]:
%%shell
./test1

CalledProcessError: ignored